In [6]:
import os

import keras.models
import trimesh
import random
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from PIL import Image
from random import seed
from random import random

import keras

In [7]:
def read_off(file):
    return trimesh.load(file).sample(8192)


def normalize(train_data):
    current = 0
    normalizedSize = 240
    new_train_data = []

    maxXReal = 0
    maxYReal = 0
    maxZReal = 0

    for model in train_data:
        minX = 123123123
        minY = 123123123
        minZ = 123123123
        maxX = -123123123
        maxY = -123123123
        maxZ = -123123123

        for point in model[1]:
            minX = min(minX, point[0])
            minY = min(minY, point[1])
            minZ = min(minZ, point[2])
            maxX = max(maxX, point[0])
            maxY = max(maxY, point[1])
            maxZ = max(maxZ, point[2])

        result = []

        if maxX - minX >= maxY - minY and maxX - minX >= maxZ - minZ:
            divRatio = maxX - minX
        elif maxY - minY >= maxX - minX and maxY - minY >= maxZ - minZ:
            divRatio = maxY - minY
        else:
            divRatio = maxZ - minZ

        for point in model[1]:
            newX = ((point[0] - minX) / divRatio) * normalizedSize
            newY = ((point[1] - minY) / divRatio) * normalizedSize
            newZ = ((point[2] - minZ) / (maxZ - minZ)) * normalizedSize

            maxXReal = max(maxXReal, newX)
            maxYReal = max(maxYReal, newY)
            maxZReal = max(maxZReal, newZ)

            result.append(np.array([newX, newY, newZ]))

            # print(str(point[0]) + " + " + str(point[1]) + " + " + str(point[2]))

        new_train_data.append((model[0], np.array(result)))

        current += 1

        if current % 100 == 0:
            print(current)

        # print(str(minX) + " + " + str(maxX))
    return new_train_data


def changeView(data):
    current = 0
    normalizedSize = 240
    images = []

    for model in data:
        slices = np.zeros((normalizedSize*4, normalizedSize*4), 'uint8')
        for point in model[1]:
            # point_slice = int((point[2] - 0.001) / 20)
            # i = int(point_slice / 4)
            # j = int(point_slice % 4)
            # #targetI = int(point[0] - 0.01 + i * normalizedSize)
            # #targetJ = int(point[1] - 0.01 + j * normalizedSize)
            # for nextI in range(3):
            #     for nextJ in range(3):
            #         goI = int(point[0] + nextI - 1 - 0.01)
            #         goJ = int(point[1] + nextJ - 1 - 0.01)
            #
            #         if goI < 0 or goI >= normalizedSize or goJ < 0 or goJ >= normalizedSize:
            #             continue
            #
            #         slices[int(point[0] - 0.01), int(point[1] - 0.01), i] |= (3 << (2 * j))
            point_slice = int((point[2] - 0.001) / 15)
            i = int(point_slice / 4)
            j = int(point_slice % 4)
            targetI = int(point[0] - 0.01 + i * normalizedSize)
            targetJ = int(point[1] - 0.01 + j * normalizedSize)
            for nextI in range(3):
                for nextJ in range(3):
                    goI = targetI + nextI - 1
                    goJ = targetJ + nextJ - 1

                    if goI < 0 or goI >= 4*normalizedSize or goJ < 0 or goJ >= 4*normalizedSize:
                        continue

                    slices[goI, goJ] = 1


        images.append((model[0], slices))
        current += 1
        if current % 100 == 0:
            print(current)

    return images

In [8]:
index = int(random() * len(test_images))
print(test_images[index][0])
# img = Image.fromarray(test_images[index][1])
# img.save('myimg.jpeg')
plt.imshow(test_images[index][1],cmap='grey_r')
plt.show()

NameError: name 'test_images' is not defined

In [10]:
categories = ["bathtub", "bed", "chair", "desk", "dresser", "monitor", "night_stand", "sofa", "table", "toilet"]

train_data = []
test_data = []

path = 'ModelNet10/'

current = 0

for category in categories:

    files_train = os.listdir(path + category + "/train")

    for f in files_train:
        train_data.append((category, read_off(path + category + "/train/" + f)))
        current += 1
        if current % 100 == 0:
            print(current)

    files_test = os.listdir(path + category + "/test")

    for f in files_test:
        test_data.append((category, read_off(path + category + "/test/" + f)))
        current += 1
        if current % 100 == 0:
            print(current)



100
200
300
400
500
600
700


FileNotFoundError: [Errno 2] No such file or directory: 'ModelNet10/chair/train'

In [11]:
train_data = normalize(train_data)
test_data = normalize(test_data)

100
200
300
400
500
100


In [12]:
train_images = changeView(train_data)
test_images = changeView(test_data)

# # %%
# random_index = random.randint(0, len(train_images))
# print(train_data[random_index][0])
#
# # %%
# fig = plt.figure(figsize=(5, 5))
# ax = fig.add_subplot(111, projection="3d")
# ax.set_xlim([0, maxXReal])
# ax.set_ylim([0, maxYReal])
# ax.set_zlim([0, maxZReal])
# ax.scatter(train_data[random_index][1][:, 0], train_data[random_index][1][:, 1], train_data[random_index][1][:, 2])
# ax.view_init(30, 200)
#
# # %%
# plt.imshow(train_images[random_index][1], cmap='gray_r')
# plt.show()

100
200
300
400
500
100


In [13]:
train_final = np.array([i[1] for i in train_images])
train_labels = np.array([categories.index(l) for l in [i[0] for i in train_images]])

In [14]:
model = tf.keras.Sequential([
    # tf.keras.layers.Conv2D(60, kernel_size=(3, 3), input_shape=(240*4, 240*4, 1)),
    # tf.keras.layers.MaxPool2D(pool_size=(8, 8)),
    # tf.keras.layers.Conv2D(60, kernel_size=(3, 3)),
    # tf.keras.layers.MaxPool2D(pool_size=(8, 8)), # !!! 2 2
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dropout(0.7),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    #tf.keras.layers.Dense(512, activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    #tf.keras.layers.Dense(256, activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

2022-01-06 01:23:15.735138: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-06 01:23:15.735682: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-06 01:23:15.736306: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-01-06 01:23:15.737295: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-01-06 01:23:15.737497: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [15]:
model.fit(train_final, train_labels, epochs=4)

Epoch 1/4


/home/cgis/ThreeObjectRecognition/research/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


18/18 [==============================] - 4s 174ms/step - loss: 0.6859 - accuracy: 0.7242
Epoch 2/4
18/18 [==============================] - 3s 169ms/step - loss: 0.0806 - accuracy: 0.9680
Epoch 3/4
18/18 [==============================] - 3s 167ms/step - loss: 0.0170 - accuracy: 0.9964
Epoch 4/4
18/18 [==============================] - 3s 166ms/step - loss: 0.0066 - accuracy: 0.9964


In [16]:
test_loss, test_acc = model.evaluate(test_final, test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

NameError: name 'test_final' is not defined

In [ ]:
model.save("first.h5")
del model

In [18]:
i = 0

In [17]:
while 1:
    i += 1
    print(i)
    print(train_final.shape, train_labels.shape)
    model.fit(train_final, train_labels, epochs=1)
    test_loss, test_acc = model.evaluate(test_final, test_labels, verbose=2)
    print('Test accuracy:', test_acc, '\n')

NameError: name 'i' is not defined

In [ ]:
test_loss, test_acc = model.evaluate(test_final, test_labels, verbose=2)
print('Test accuracy:', test_acc, '\n')

In [ ]:
model.save("third.h5")
np.save('train_final_third.npy', train_final)
np.save('test_final_third.npy', test_final)
np.save('train_labels_third.npy', train_labels)
np.save('test_labels_third.npy', test_labels)

In [ ]:
model = keras.models.load_model("third.h5")
train_final = np.load("train_final_third.npy")
test_final = np.load("test_final_third.npy")
train_labels = np.load("train_labels_third.npy")
test_labels = np.load("test_labels_third.npy")